## **Formal Definition of Diffrenetial Privacy**

* Local DP
* Differencing Attack
* Basic queries
* Sensitivity
* Diffrenrial Privacy Definition

Its a constraint so that you can analyze a query with noise and know wheather or not this query and noise is leaking too much of information.

To measure threshold of leakeage 
1.  Epsilon 
2.  Delta

"Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy.

However, alternatively we can add noise AFTER data has been aggregated by a function. In "Global Differentail Privacy" instead of adding noise to individua datapoint,it applies noise to the output of a function on datapoints.The advantage is that , in this kind of noise can allow for similar levels of protection with a lower affect on accuracy.

However, participants must be able to trust that no-one looked at their datapoints before the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.




In [2]:
# Database

import torch

num_entries = 5000

db = torch.rand(num_entries) > 0.5

db

# Create a function - To Remove Index
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index],db[remove_index+1:]))

# Create Function 2 to iterate through db and create parallel db

def get_parallel_dbs(db):
    parallel_dbs = list()
    for i in range(len(db)):
        pdb = get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs

# Function 3 to 
def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

#function sensitivity
def sensitivity(query, n_entries = 1000):
    #Initialize a database of correct size and all parallel database
    db , pdbs = create_db_and_parallels(n_entries)
    full_db_result = query(db)
    #Run the Query over all the databases
    max_distance = 0 
    for pdb in pdbs:
        pdb_result = query(pdb)
        # comapre paralled db and full db
        db_distance = torch.abs(pdb_result - full_db_result)
        if(db_distance > max_distance):
            max_distance = db_distance
    #Return the sensitivity
    return max_distance

In [5]:

db, pdbs = create_db_and_parallels(100)

def query(db):
    noise = 0.2
    # Local Dp 
    return torch.sum(db.float() + noise)   
   # return torch.sum(db.float())  + noise    # Global DP

def M(db):
    query(db) + noise

query(db)

tensor(73.0000)

In [6]:

db, pdbs = create_db_and_parallels(100)

def query(db):
    # Local Dp 
    # return torch.sum(db.float() + noise)
    noise = 0.2
    return torch.sum(db.float())  + noise    # Global DP

def M(db):
    query(db) + noise

query(db)

tensor(43.2000)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add 
- Laplacian Noise or 
- Gaussian Noise.

However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

### Differential Privacy - A randomized algorithm $M$ with domain N |X| is (ε, δ)-differentially private if for all $S$ ⊆ Range($M$) and for all $x, y$ ∈ N |X| such that $∥x − y$∥1 ≤ 1: 

   **Pr[M(x) ∈ S] ≤ exp(ε) Pr[M(y) ∈ S] + δ**

"The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf

This definition does not create differential privacy, instead it is a measure of how much privacy is afforded by a query $M$. Specifically, it's a comparison between running the query $M$ on a database $(x)$ and a parallel database $(y)$. As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database $(x)$ and the same query on database $(y)$ will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

#### How do we actually use epsilon and delta? 

We will see how to take a query and add **Randomized Mechanism**(A function with random noise added to its input , output and/or inner workings)

#### **Epsilon**
Let's unpack the intuition of this for a moment.

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

Delta
Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.


### **Lesson: How To Add Noise for Global Differential Privacy**
In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy.

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

#### How much noise should we add to the query?
The amount of noise necessary to add to the output is the query is a function of 4 things
1.  Type of NOise (Gaussian/Laplacian) 
    - Here we will be using Laplacian
2. Sensitivy of Query
    - Sensitivity of the query that we are using to query the database.
3. Desired Epsilon (ε)
4. Desired Delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. 

Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

**In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon.** Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

#### Querying Repeatedly
if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

## Project: Create a Differentially Private Query
In this project, I want you to take what you learned in the previous lesson and 
* create a query function which sums over the database and 
* adds just the right amount of noise such that it satisfies an epsilon constraint.(use random.laplace ) 
* beta = sensitivity(query) / epsilon
* Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [23]:

epsilon = 0.0001

In [8]:
import numpy as np

In [9]:
db, pdbs = create_db_and_parallels(100)

In [11]:
db

tensor([1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 0, 0], dtype=torch.uint8)

In [12]:
def sum_query(db):
    return db.sum()

In [13]:
def laplacian_mechanism(db, query, sensitivity):
    
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    
    return query(db) + noise

In [14]:
sum_query(db)

tensor(47)

In [16]:
laplacian_mechanism(db, sum_query, 1)

tensor([-311.2988], dtype=torch.float64)

In [17]:
# for mean query
def mean_query(db):
    return torch.mean(db.float())

In [24]:
laplacian_mechanism(db, mean_query, 1/100)

tensor([0.0630], dtype=torch.float64)